In [1]:
import os
import ee
import geetools
import geemap
import mgwr.sel_bw
import numpy
import pandas
import geopandas
import mgwr
import numpy as np
import time
from osgeo import gdal
from osgeo import osr
import fiona
import matplotlib.pyplot as plt
import folium
from folium import plugins
from IPython.display import Image
from IPython.core.display import display, HTML
import patsy


In [2]:
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}


def add_ee_layer(self, ee_object, vis_params, name):
    try:
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
                data=ee_object.getInfo(),
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Google Earth Engine',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)

    except:
        print("Could not display {}".format(name))


# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWgiWsAukwLIf3w2AzxDbXP6K__2IuFfQavDdxWSCofAjqp4O_EUsDs

Successfully saved authorization token.


In [18]:
AOI = geemap.shp_to_ee("C:/Users/285908i/Desktop/GEE_GWR/Bound.shp")
shp = fiona.open("C:/Users/285908i/Desktop/GEE_GWR/Bound.shp")
geom = shp.bounds
geom_list = list(geom)
print(geom_list)
area = ee.Geometry.Rectangle(geom_list)

START_DATE = '2021-01-01'
END_DATE = '2021-06-30'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50


def get_s2_sr_cld_col(AOI, START_DATE, END_DATE):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(AOI)
        .filterDate(START_DATE, END_DATE)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))
    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(AOI)
        .filterDate(START_DATE, END_DATE))
    clip_cloudless_col = s2_cloudless_col.map(lambda image: image.clip(AOI))
    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)


def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)
    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')
    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));
    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))
    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)
    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)
    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)
    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))
    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)



s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)

img = s2_sr_cld_col_eval_disp.mosaic()

    # Subset layers and prepare them for display.
clouds = img.select('clouds').selfMask()
shadows = img.select('shadows').selfMask()
dark_pixels = img.select('dark_pixels').selfMask()
probability = img.select('probability')
cloudmask = img.select('cloudmask').selfMask()
cloud_transform = img.select('cloud_transform')

s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()
    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)


s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())




NDVI  = ee.Image(s2_sr_median.normalizedDifference(['B8', 'B4'])).rename(["NDVI"])
NDVI_clip = ee.Image(NDVI).clip(area)

[116.71355928442438, -33.38776926871642, 117.03743005306924, -33.17665168654703]


In [21]:
vis_params = {
  'min': 0.07,
  'max': 0.45,
  'palette': ['green','yellow','red']}

# Create a folium map object.
my_map = folium.Map(location=[-31, 117], zoom_start=10, height=10000)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

my_map.add_ee_layer(NDVI.updateMask(NDVI.gt(0)), vis_params, 'NDVI')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

In [22]:
chirps = (ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')
    .filterBounds(AOI)
    .filterDate(START_DATE, END_DATE))

rain_mean = chirps.sum()
rain_clip = ee.Image(rain_mean).clip(area)

rain_vis_params = {
  'min': 0,
  'max': 1000,
  'palette': ['orange','green','blue']}

# Create a folium map object.
rain_map = folium.Map(location=[-31, 117], zoom_start=10, height=10000)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)


rain_map.add_ee_layer(rain_clip.updateMask(rain_clip.gt(0)), rain_vis_params, 'rain')


rain_map.add_child(folium.LayerControl())


plugins.Fullscreen().add_to(rain_map)

# Display the map.
display(rain_map)


ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.clip",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  [
                    116.71355928442438,
                    -33.17665168654703
                  ],
                  [
                    116.71355928442438,
                    -33.38776926871642
                  ],
                  [
                    117.03743005306924,
                    -33.38776926871642
                  ],
                  [
                    117.03743005306924,
                    -33.17665168654703
                  ]
                ]
              ]
            },
            "evenOdd": {
              "constantValue": true
            }
          }
        }
      },
      "input": {
        "functionInvocationValue":

In [26]:
latlon = ee.Image.pixelLonLat().addBands(NDVI_clip)
rain = ee.Image.pixelLonLat().addBands(rain_clip)

latlon = latlon.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=area,
  scale=50);

rain = rain.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=area,
  scale=50);

rain_np = np.array((ee.Array(rain.get('precipitation')).getInfo()))
data = np.array((ee.Array(latlon.get("NDVI")).getInfo()))
lats = np.array((ee.Array(latlon.get("latitude")).getInfo()))
lons = np.array((ee.Array(latlon.get("longitude")).getInfo()))

In [27]:
df = pandas.DataFrame(data=[data, rain_np,lats,lons]).T
df.columns = ['NDVI','rain','lat','lon']
coords = numpy.column_stack([df[['lon','lat']]])
y,X = patsy.dmatrices('NDVI ~ 0 + rain', data = df)

bw = mgwr.sel_bw.Sel_BW(coords,
                        numpy.asarray(y),
                        numpy.asarray(X),
                        fixed = False,
                        spherical= True)

bw_out = bw.search()

print(bw_out)

model = mgwr.gwr.GWR(coords,
                     numpy.asarray(y),
                     numpy.asarray(X),
                     bw.bw[0])

results = model.fit()

print(results.summary())

c:\python39\lib\site-packages\spglm\iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=6.13513e-21): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)


LinAlgError: Matrix is singular.